In [19]:
# Standard imports
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt


# Astropy imports
from astropy.io import fits
from astropy import units as u
from astropy.table import Table
from astropy.units import Quantity
from astropy import constants as const
import corner
from astropy.table import vstack

# SLSim imports
import slsim.Pipelines as pipelines

from hierarc.Sampling.ParamManager.cosmo_param import CosmoParam

## Generate SLSim Catalog

In [ ]:
# define a cosmology
cosmology = "FwCDM"  # Flat wCDM cosmology
# other options are: "FLCDM FwCDM", "w0waCDM", "oLCDM"
kwargs_cosmo_true = {"h0": 70, "om": 0.3, "w": -1}  # cosmological model of the forecast

# create astropy.cosmology instance of input cosmology
cosmo_param = CosmoParam(cosmology=cosmology)
cosmo_true = cosmo_param.cosmo(kwargs_cosmo_true)

# define a sky area
sky_area_galaxies = Quantity(value=20, unit="deg2")
scale_factor = 5 # to increase sky area for better statistics
final_sky_area = sky_area_galaxies * scale_factor

# Generate galaxy population using skypy pipeline.
blue_galaxies = Table()
red_galaxies = Table()
for i in tqdm(range(scale_factor), desc="Generating galaxy catalogs"):
    galaxy_simulation_pipeline = pipelines.SkyPyPipeline(
        skypy_config=None, sky_area=sky_area_galaxies, filters=None, cosmo=cosmo_true
    )
    blue_galaxies = vstack([blue_galaxies, galaxy_simulation_pipeline.blue_galaxies])
    red_galaxies = vstack([red_galaxies, galaxy_simulation_pipeline.red_galaxies])


Generating galaxy catalogs:  20%|██        | 1/5 [07:37<30:31, 457.86s/it]

In [ ]:
# save the catalogs
blue_galaxies.write(
    f"../data/skypy_galaxy_catalogs/SKYPY_BLUE_GALAXIES_{final_sky_area.value}_SQDEG.fits",
    format="fits", overwrite=True
)
red_galaxies.write(
    f"../data/skypy_galaxy_catalogs/SKYPY_RED_GALAXIES_{final_sky_area.value}_SQDEG.fits",
    format="fits", overwrite=True
)

In [ ]:
# galaxy_simulation_pipeline.blue_galaxies.write(
#     "../data/skypy_galaxy_catalogs/SKYPY_BLUE_GALAXIES_10_SQDEG.fits",
#     format="fits", overwrite=True
# )